In [67]:
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe # <--- Import this
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe
from google.cloud import bigquery

In [68]:
# --- 1. CONFIGURATION ---
CREDENTIALS_FILE = 'gen-lang-client-0209575391-96d90a513b0b.json'
SHEET_ID = '1hHV98ZAqng6ogy48iKDTP_tMVP8siwzgB8ybLSq40kU'
WORKSHEET_NAME = 'Transformed'

## READ DATA

In [69]:
# 1. Read the Parquet file
df = pd.read_parquet("daily_funding.parquet")
print(f"Successfully read parquet file with {len(df)} rows.")

# 2. String Normalization
df["brand"] = df["brand"].astype(str).str.upper().str.strip()

# 3. Ensure Dates are standard Datetime objects (Fixes filtering issues)
# Even if parquet stores them correctly, this ensures they are datetime64[ns]
# which allows for direct comparison like: df['Date'] >= '2025-11-01'
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])

Successfully read parquet file with 1126947 rows.


In [70]:
df["Max_InsertedAt"].max()

Timestamp('2026-02-16 08:34:46.304597+0000', tz='UTC')

In [71]:
client = bigquery.Client(project = 'kz-dp-prod')

# ... [Keep your existing SQL query exactly as is] ...
sql_query = """
SELECT
  Brand AS brand,
  Sub_group AS sub_group,
  CASE
    WHEN Whitelabel = 'KZ' THEN 'KZO'
    ELSE 'KZP'
  END AS whitelabel
FROM `kz-dp-prod.MAPPING.brand_whitelabel_country_folderid_mapping_tbl`;
"""

dim_brand_v2 = client.query(sql_query).to_dataframe()

d:\Projects\KZG\PGW-Health-Pipeline\.venv\Lib\site-packages\google\auth\_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [72]:
dim_brand_v2["brand"] = dim_brand_v2["brand"].str.strip().str.upper()

In [73]:
# dim_brand_v2 = pd.read_csv("mapping_brand_v2.csv") 
# dim_brand_v2 = dim_brand_v2[["brand","sub_group", "whitelabel"]]

In [74]:
dim_brand_v2.columns = ["brand", "account_group", "group_re"]

In [75]:
# dim_brand_v2.loc[dim_brand_v2["group_re"] == 'KZ', 'group_re'] = 'KZO'
# dim_brand_v2.loc[dim_brand_v2["group_re"] != 'KZO', 'group_re'] = 'KZP'
# dim_brand_v2.loc[(dim_brand_v2["group_re"] != 'KZP') & (dim_brand_v2["group_re"] != 'KZo'), 'group_re'] = 'KZO'

In [76]:
dim_brand_v2["brand"] = dim_brand_v2["brand"].str.upper().str.strip()

In [77]:
df[((df["status"] == "completed") | (df["status"] == "timeout") | (df["status"] == "error")) & (df["Country"] == "BD")]

,Date,providerKey,method,channel_type,type,reqCurrency,account_group,group_re,Country,status,...,Count_01m31s_to_02m00s,Count_02m01s_to_03m00s,Count_03m00s_Above,Count_03m00s_Below,Count_03m31s_to_05m00s,Count_05m00s_to_10m00s,Count_10m00s_Above,providerName,channel_main,brand


In [78]:
df = df[[i for i in df.columns if i not in ["group_re", "account_group"]]].merge(dim_brand_v2, on = "brand", how = "left")

In [79]:
df_deposit = df[df['type'] == 'DEPOSIT']

In [80]:
df_deposit

,Date,providerKey,method,channel_type,type,reqCurrency,Country,status,Hour,Count,...,Count_03m00s_Above,Count_03m00s_Below,Count_03m31s_to_05m00s,Count_05m00s_to_10m00s,Count_10m00s_Above,providerName,channel_main,brand,account_group,group_re
0,2025-12-16,mmpay,mmpay/gcash-direct,gcash-direct,DEPOSIT,PHP,Philippines,timeout,04:00 - 04:59,11,...,0,0,0,0,0,mmpay,gcash,WINMAYA,PH96G1,KZP
1,2025-12-07,jompay,jompay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,timeout,05:00 - 05:59,6,...,0,0,0,0,0,jompay,qr,NOO66,WDB1,KZP
2,2026-02-16,vcpay-ph-native,vcpay-ph-native/gcash-native-direct,gcash-native-direct,DEPOSIT,PHP,Philippines,completed,14:00 - 14:59,28,...,0,28,0,0,0,vcpay-ph-native,gcash,WINLUCK,PHKZG1,KZO
3,2025-12-27,swpay,swpay/jazz-cash,jazz-cash,DEPOSIT,PKR,Pakistan,timeout,13:00 - 13:59,22,...,0,0,0,0,0,swpay,jazz,WOWPK,KZG1,KZO
4,2025-12-16,rmpay,rmpay/gcash-qr,gcash-qr,DEPOSIT,PHP,Philippines,timeout,22:00 - 22:59,31,...,0,0,0,0,0,rmpay,gcash,WINMAYA,PH96G1,KZP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192217,2025-12-01,wepay,wepay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,timeout,21:00 - 21:59,1,...,0,0,0,0,0,wepay,qr,KISS39,THKZG2,KZO
1192218,2025-12-02,wepay,wepay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,timeout,09:00 - 09:59,2,...,0,0,0,0,0,wepay,qr,77AC,BLG1,KZP
1192219,2025-12-10,wepay,wepay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,timeout,11:00 - 11:59,1,...,0,0,0,0,0,wepay,qr,POP369,THKZG2,KZO
1192220,2026-02-13,wepay,wepay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,error,14:00 - 14:59,1,...,0,0,0,0,0,wepay,qr,BIG188,THKZG2,KZO


In [81]:
df_withdraw = df[df['type'] == 'WITHDRAW']

## DEPOSIT DATA

In [82]:
df_deposit = df_deposit[df_deposit["Date"] >= '2025-11-01']

In [83]:
df_deposit

,Date,providerKey,method,channel_type,type,reqCurrency,Country,status,Hour,Count,...,Count_03m00s_Above,Count_03m00s_Below,Count_03m31s_to_05m00s,Count_05m00s_to_10m00s,Count_10m00s_Above,providerName,channel_main,brand,account_group,group_re
0,2025-12-16,mmpay,mmpay/gcash-direct,gcash-direct,DEPOSIT,PHP,Philippines,timeout,04:00 - 04:59,11,...,0,0,0,0,0,mmpay,gcash,WINMAYA,PH96G1,KZP
1,2025-12-07,jompay,jompay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,timeout,05:00 - 05:59,6,...,0,0,0,0,0,jompay,qr,NOO66,WDB1,KZP
2,2026-02-16,vcpay-ph-native,vcpay-ph-native/gcash-native-direct,gcash-native-direct,DEPOSIT,PHP,Philippines,completed,14:00 - 14:59,28,...,0,28,0,0,0,vcpay-ph-native,gcash,WINLUCK,PHKZG1,KZO
3,2025-12-27,swpay,swpay/jazz-cash,jazz-cash,DEPOSIT,PKR,Pakistan,timeout,13:00 - 13:59,22,...,0,0,0,0,0,swpay,jazz,WOWPK,KZG1,KZO
4,2025-12-16,rmpay,rmpay/gcash-qr,gcash-qr,DEPOSIT,PHP,Philippines,timeout,22:00 - 22:59,31,...,0,0,0,0,0,rmpay,gcash,WINMAYA,PH96G1,KZP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192217,2025-12-01,wepay,wepay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,timeout,21:00 - 21:59,1,...,0,0,0,0,0,wepay,qr,KISS39,THKZG2,KZO
1192218,2025-12-02,wepay,wepay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,timeout,09:00 - 09:59,2,...,0,0,0,0,0,wepay,qr,77AC,BLG1,KZP
1192219,2025-12-10,wepay,wepay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,timeout,11:00 - 11:59,1,...,0,0,0,0,0,wepay,qr,POP369,THKZG2,KZO
1192220,2026-02-13,wepay,wepay/qr-code-native,qr-code-native,DEPOSIT,THB,Thailand,error,14:00 - 14:59,1,...,0,0,0,0,0,wepay,qr,BIG188,THKZG2,KZO


In [84]:
# import pandas as pd
# import gspread
# from gspread_dataframe import set_with_dataframe
# from google.oauth2.service_account import Credentials

# # --- 1. CONFIGURATION ---
# CREDENTIALS_FILE = 'gen-lang-client-0209575391-96d90a513b0b.json'
# SHEET_ID = '1Nq9u4bg0tvLnUutVh2TcxXOxe-G2E65kxy_pbJ8pce4'
# WORKSHEET_NAME = 'Deposit Data'

# def write_df_to_gsheet(dataframe):
#     # --- 2. AUTHENTICATION ---
#     # Define the scope (permissions) required
#     scopes = [
#         'https://www.googleapis.com/auth/spreadsheets',
#         'https://www.googleapis.com/auth/drive'
#     ]
    
#     # Authenticate using the JSON key file
#     credentials = Credentials.from_service_account_file(
#         CREDENTIALS_FILE, 
#         scopes=scopes
#     )
#     gc = gspread.authorize(credentials)

#     # --- 3. CONNECT TO SHEET ---
#     print(f"Opening sheet ID: {SHEET_ID}...")
#     sh = gc.open_by_key(SHEET_ID)
    
#     try:
#         worksheet = sh.worksheet(WORKSHEET_NAME)
#     except gspread.WorksheetNotFound:
#         print(f"Worksheet '{WORKSHEET_NAME}' not found. Creating it...")
#         worksheet = sh.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

#     # --- 4. WRITE DATA ---
#     print("Clearing old data...")
#     worksheet.clear()  # Removes old data so rows don't overlap
    
#     print(f"Writing {len(dataframe)} rows to Google Sheets...")
#     # 'set_with_dataframe' handles headers and NaN values automatically
#     set_with_dataframe(worksheet, dataframe)
    
#     print("Done!")

# # --- Usage ---
# if __name__ == "__main__":
#     # Assuming 'df' exists from your previous code
#     # If not, uncomment the line below to test with dummy data:
#     # df = pd.DataFrame({'Date': ['2023-01-01'], 'Amount': [100]})
    
#     write_df_to_gsheet(df_deposit)

In [85]:
df_deposit.loc[df_deposit["status"] == "completed", "Count Success"] = df_deposit["Count"]

df_deposit.loc[df_deposit["status"].str.contains("error", na=False), "Count Error"] = df_deposit["Count"]

df_deposit.loc[df_deposit["status"].str.contains("timeout", na=False), "Count Timeout"] = df_deposit["Count"]

In [86]:
df_deposit.loc[df_deposit["status"].str.contains("error", na=False)].shape[0] + df_deposit.loc[df_deposit["status"].str.contains("timeout", na=False)].shape[0] + df_deposit.loc[df_deposit["status"] == "completed", "Count Success"].shape[0]

874598

In [87]:
df_deposit["Count"].sum()


np.int64(27585221)

In [88]:
# 1. Define your columns
group_cols = ["Date", "providerName", "channel_type", "type", "Country", "account_group", "group_re", "Hour"]

sum_cols = [
    "Count", "Count Success", "Count Error", "Count Timeout",
    "Total_Net_Amount", 
    "winsorized_total_time_seconds", 
    'Count_01m30s_Below', 'Count_01m31s_to_02m00s',
       'Count_02m01s_to_03m00s', 'Count_03m00s_Above'
]

# 2. Perform Group By and Sum
df_grouped_dep = df_deposit.groupby(group_cols)[sum_cols].sum().reset_index()

# 3. (Optional) Inspect the result
print(df_grouped_dep.head())

        Date providerName          channel_type     type   Country  \
0 2025-11-12    toppay-mx         bank-transfer  DEPOSIT    Mexico   
1 2025-11-13        hspay  bank-transfer-native  DEPOSIT  Thailand   
2 2025-11-20        hspay  bank-transfer-native  DEPOSIT  Thailand   
3 2025-11-26        hspay  bank-transfer-native  DEPOSIT  Thailand   
4 2025-11-27        hspay  bank-transfer-native  DEPOSIT  Thailand   

  account_group group_re           Hour  Count  Count Success  Count Error  \
0          KZG1      KZO  00:00 - 00:59      1              1            0   
1          WDB1      KZP  20:00 - 20:59      1              1            0   
2        THKZG2      KZO  07:00 - 07:59      1              1            0   
3        THKZG1      KZO  05:00 - 05:59      1              1            0   
4          BLG1      KZP  22:00 - 22:59      1              1            0   

   Count Timeout  Total_Net_Amount  winsorized_total_time_seconds  \
0              0              10.0       

In [89]:
import pandas as pd
from datetime import timedelta

# --- 1. PREPARE DATA & INDICES ---
# Assumes df_grouped has [Date, Hour, providerName, etc., Count, NetAmount...]

df_grouped_dep['Date'] = pd.to_datetime(df_grouped_dep['Date'])
df_grouped_dep['Week_Index'] = df_grouped_dep['Date'].dt.isocalendar().week.astype(int)

# Helper: Day of Week (0=Mon, 6=Sun)
df_grouped_dep['DayOfWeek'] = df_grouped_dep['Date'].dt.dayofweek

# --- 2. DETERMINE THE CUTOFF (Based on Current Week) ---
print("Identifying Cutoff from Current Week...")

# Identify the Current Week (Highest Index)
current_week_idx = df_grouped_dep['Week_Index'].max()
current_week_data = df_grouped_dep[df_grouped_dep['Week_Index'] == current_week_idx]

if not current_week_data.empty:
    # 1. How far into the week are we? (e.g., Today is Monday = 0)
    cutoff_day_idx = current_week_data['DayOfWeek'].max()
    
    # 2. What is the latest hour on that specific day?
    cutoff_hour = current_week_data.loc[
        current_week_data['DayOfWeek'] == cutoff_day_idx, 'Hour'
    ].max()
    
    print(f" -> Cutoff Point: Day {cutoff_day_idx} (Mon=0) at {cutoff_hour}")
else:
    # Fallback if current week is empty
    cutoff_day_idx = 6 # Sunday
    cutoff_hour = "23:59 - 23:59"
    print(" -> No current data. defaulting to full week.")

# --- 3. CREATE THE 'MARK' COLUMN ---
print("Tagging rows with 'pacing_mark'...")

def get_pacing_mark(row):
    # Rule 1: Always include the current week (it is the reference)
    if row['Week_Index'] == current_week_idx:
        return 'Included'
    
    # Rule 2: For historical weeks, compare Day & Hour
    # A. If the day is EARLIER in the week (e.g. Current is Wed, Row is Mon) -> Keep
    if row['DayOfWeek'] < cutoff_day_idx:
        return 'Included'
    
    # B. If it's the SAME Day, check the Hour
    if row['DayOfWeek'] == cutoff_day_idx:
        if row['Hour'] <= cutoff_hour:
            return 'Included'
        else:
            return 'Excluded' # Same day, but later hour
            
    # C. If the day is LATER in the week (e.g. Current is Mon, Row is Tue) -> Exclude
    if row['DayOfWeek'] > cutoff_day_idx:
        return 'Excluded'

    return 'Excluded'

# Apply the logic
df_grouped_dep['pacing_mark'] = df_grouped_dep.apply(get_pacing_mark, axis=1)

# --- 4. GENERATE LABELS ---
print("Generating Week Labels...")
# Week start (Monday)
df_grouped_dep['Week_Start'] = df_grouped_dep['Date'] - pd.to_timedelta(df_grouped_dep['Date'].dt.dayofweek, unit='D')

# Week end (Sunday)
df_grouped_dep['Week_End'] = df_grouped_dep['Week_Start'] + pd.to_timedelta(6, unit='D')

# Use Week_End for label
df_grouped_dep['Week_Label'] = (
    df_grouped_dep['Week_End'].dt.day.astype(str) + " " +
    df_grouped_dep['Week_End'].dt.strftime('%b %y')
)

# --- 5. AGGREGATE (INCLUDE 'pacing_mark' IN GROUP BY) ---
print("Aggregating...")

final_group_cols = [
    "Week_Label", 
    "Week_Index", 
    "pacing_mark", # <--- CRITICAL: Keep this so you can filter in Looker
    "providerName", "channel_type", "type", "Country", "account_group", "group_re"
]
sum_cols = [
    "Count", "Count Success", "Count Error", "Count Timeout",
    "Total_Net_Amount", 
    "winsorized_total_time_seconds", 
    'Count_01m30s_Below', 'Count_01m31s_to_02m00s',
       'Count_02m01s_to_03m00s', 'Count_03m00s_Above'
]

df_final_dep = df_grouped_dep.groupby(final_group_cols)[sum_cols].sum().reset_index()

# Sort DESC so Week 49 is top
df_final_dep = df_final_dep.sort_values(by=['Week_Index', 'pacing_mark'], ascending=[False, True])

# print(df_final[['Week_Label', 'pacing_mark', 'Count']].head())

# write_df_to_gsheet(df_final)



Identifying Cutoff from Current Week...
 -> Cutoff Point: Day 6 (Mon=0) at 23:00 - 23:59
Tagging rows with 'pacing_mark'...
Generating Week Labels...
Aggregating...


In [90]:
df_final_dep.loc[df_final_dep["Week_Label"].str.contains("26"), "Week_Index"] += 52

In [91]:
df_final_dep

,Week_Label,Week_Index,pacing_mark,providerName,channel_type,type,Country,account_group,group_re,Count,Count Success,Count Error,Count Timeout,Total_Net_Amount,winsorized_total_time_seconds,Count_01m30s_Below,Count_01m31s_to_02m00s,Count_02m01s_to_03m00s,Count_03m00s_Above
1330,28 Dec 25,52,Included,aipay,qr-code,DEPOSIT,Thailand,96G1,KZP,1306,979,27,300,1074492.00,77288.0,771,101,59,48
1331,28 Dec 25,52,Included,aipay,qr-code,DEPOSIT,Thailand,KZG1,KZP,1222,850,16,356,434389.00,110474.0,374,210,168,98
1332,28 Dec 25,52,Included,aipay,qr-code,DEPOSIT,Thailand,Others,KZP,1641,1206,44,391,599058.00,129771.0,606,303,221,76
1333,28 Dec 25,52,Included,aipay,qr-code,DEPOSIT,Thailand,THKZG1,KZO,6795,5183,107,1505,2549155.39,600258.0,2480,1274,1019,410
1334,28 Dec 25,52,Included,aipay,qr-code,DEPOSIT,Thailand,THKZG2,KZO,6413,4899,206,1308,1987161.60,523034.0,2598,1121,853,327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,4 Jan 26,53,Included,xqpay-bd,nagad,DEPOSIT,Bangladesh,KZG1,KZO,4881,3472,12,1397,1622905.00,239390.0,2923,311,150,88
1698,4 Jan 26,53,Included,xqpay-bd,rocket,DEPOSIT,Bangladesh,KZG1,KZO,184,90,0,94,47265.00,12501.0,41,29,10,10
1699,4 Jan 26,53,Included,ydpay-mx,clabe,DEPOSIT,Mexico,KZG1,KZO,446,338,3,105,27961.67,45984.0,137,64,58,79
1700,4 Jan 26,53,Included,yzpay-th,bank-transfer,DEPOSIT,Thailand,BLG1,KZP,192,84,0,108,49513.07,38242.0,0,10,7,67


In [92]:
df_final_dep.columns = ['Week_Label', 'Week_Index', 'pacing_mark','providerName', 'channel_type', 'type', 'Country', 'account_group', "group_re",
        'DP Count', "Count Success", "Count Error", "Count Timeout", 'Total Amount DP',
       'Avg DP Time',     'Count_01m30s_Below', 'Count_01m31s_to_02m00s',
       'Count_02m01s_to_03m00s', 'Count_03m00s_Above']

# cond1 = df_final_dep["Week_Index"] == (df_final_dep["Week_Index"].max() - 1)
# cond2 = df_final_dep["pacing_mark"] == "Excluded"
# df_final_dep.loc[~(cond1 & cond2), "pacing_mark"] = None

df_final_dep["Week_Index"] = df_final_dep["Week_Index"] - df_final_dep["Week_Index"].max()

# df_final_dep = df_final_dep[df_final_dep["Week_Index"] != 0 ]
# df_final_dep["Week_Index"] = df_final_dep["Week_Index"] + 1
# df_final_dep = df_final_dep[df_final_dep["Week_Index"] >= -5]

In [93]:
df_final_dep

,Week_Label,Week_Index,pacing_mark,providerName,channel_type,type,Country,account_group,group_re,DP Count,Count Success,Count Error,Count Timeout,Total Amount DP,Avg DP Time,Count_01m30s_Below,Count_01m31s_to_02m00s,Count_02m01s_to_03m00s,Count_03m00s_Above
1330,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,96G1,KZP,1306,979,27,300,1074492.00,77288.0,771,101,59,48
1331,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,KZG1,KZP,1222,850,16,356,434389.00,110474.0,374,210,168,98
1332,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,Others,KZP,1641,1206,44,391,599058.00,129771.0,606,303,221,76
1333,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,THKZG1,KZO,6795,5183,107,1505,2549155.39,600258.0,2480,1274,1019,410
1334,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,THKZG2,KZO,6413,4899,206,1308,1987161.60,523034.0,2598,1121,853,327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,4 Jan 26,-7,Included,xqpay-bd,nagad,DEPOSIT,Bangladesh,KZG1,KZO,4881,3472,12,1397,1622905.00,239390.0,2923,311,150,88
1698,4 Jan 26,-7,Included,xqpay-bd,rocket,DEPOSIT,Bangladesh,KZG1,KZO,184,90,0,94,47265.00,12501.0,41,29,10,10
1699,4 Jan 26,-7,Included,ydpay-mx,clabe,DEPOSIT,Mexico,KZG1,KZO,446,338,3,105,27961.67,45984.0,137,64,58,79
1700,4 Jan 26,-7,Included,yzpay-th,bank-transfer,DEPOSIT,Thailand,BLG1,KZP,192,84,0,108,49513.07,38242.0,0,10,7,67


## WITHDRAWL DATA

In [94]:
df_final_dep

,Week_Label,Week_Index,pacing_mark,providerName,channel_type,type,Country,account_group,group_re,DP Count,Count Success,Count Error,Count Timeout,Total Amount DP,Avg DP Time,Count_01m30s_Below,Count_01m31s_to_02m00s,Count_02m01s_to_03m00s,Count_03m00s_Above
1330,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,96G1,KZP,1306,979,27,300,1074492.00,77288.0,771,101,59,48
1331,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,KZG1,KZP,1222,850,16,356,434389.00,110474.0,374,210,168,98
1332,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,Others,KZP,1641,1206,44,391,599058.00,129771.0,606,303,221,76
1333,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,THKZG1,KZO,6795,5183,107,1505,2549155.39,600258.0,2480,1274,1019,410
1334,28 Dec 25,-8,Included,aipay,qr-code,DEPOSIT,Thailand,THKZG2,KZO,6413,4899,206,1308,1987161.60,523034.0,2598,1121,853,327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,4 Jan 26,-7,Included,xqpay-bd,nagad,DEPOSIT,Bangladesh,KZG1,KZO,4881,3472,12,1397,1622905.00,239390.0,2923,311,150,88
1698,4 Jan 26,-7,Included,xqpay-bd,rocket,DEPOSIT,Bangladesh,KZG1,KZO,184,90,0,94,47265.00,12501.0,41,29,10,10
1699,4 Jan 26,-7,Included,ydpay-mx,clabe,DEPOSIT,Mexico,KZG1,KZO,446,338,3,105,27961.67,45984.0,137,64,58,79
1700,4 Jan 26,-7,Included,yzpay-th,bank-transfer,DEPOSIT,Thailand,BLG1,KZP,192,84,0,108,49513.07,38242.0,0,10,7,67


In [95]:
# import pandas as pd
# import gspread
# from gspread_dataframe import set_with_dataframe
# from google.oauth2.service_account import Credentials

# # --- 1. CONFIGURATION ---
# CREDENTIALS_FILE = 'gen-lang-client-0209575391-96d90a513b0b.json'
# SHEET_ID = '1Nq9u4bg0tvLnUutVh2TcxXOxe-G2E65kxy_pbJ8pce4'
# WORKSHEET_NAME = 'Withdrawal Data'

# def write_df_to_gsheet(dataframe):
#     # --- 2. AUTHENTICATION ---
#     # Define the scope (permissions) required
#     scopes = [
#         'https://www.googleapis.com/auth/spreadsheets',
#         'https://www.googleapis.com/auth/drive'
#     ]
    
#     # Authenticate using the JSON key file
#     credentials = Credentials.from_service_account_file(
#         CREDENTIALS_FILE, 
#         scopes=scopes
#     )
#     gc = gspread.authorize(credentials)

#     # --- 3. CONNECT TO SHEET ---
#     print(f"Opening sheet ID: {SHEET_ID}...")
#     sh = gc.open_by_key(SHEET_ID)
    
#     try:
#         worksheet = sh.worksheet(WORKSHEET_NAME)
#     except gspread.WorksheetNotFound:
#         print(f"Worksheet '{WORKSHEET_NAME}' not found. Creating it...")
#         worksheet = sh.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

#     # --- 4. WRITE DATA ---
#     print("Clearing old data...")
#     worksheet.clear()  # Removes old data so rows don't overlap
    
#     print(f"Writing {len(dataframe)} rows to Google Sheets...")
#     # 'set_with_dataframe' handles headers and NaN values automatically
#     set_with_dataframe(worksheet, dataframe)
    
#     print("Done!")

# # --- Usage ---
# if __name__ == "__main__":
#     # Assuming 'df' exists from your previous code
#     # If not, uncomment the line below to test with dummy data:
#     # df = pd.DataFrame({'Date': ['2023-01-01'], 'Amount': [100]})
    
#     write_df_to_gsheet(df_withdraw)

In [96]:
df_withdraw[["Date", "providerName", "channel_type", "type" , "Country", "account_group"]]

,Date,providerName,channel_type,type,Country,account_group
10,2026-02-13,sdpay,jazz-cash,WITHDRAW,Pakistan,KZG1
13,2026-02-15,akepay-mx,clabe,WITHDRAW,Mexico,KZG1
14,2026-02-04,vcpay-ph-native,gcash,WITHDRAW,Philippines,PHKZG1
17,2025-12-23,rmpay,gcash,WITHDRAW,Philippines,PH96G1
20,2026-01-07,tgpay-bd,bkash,WITHDRAW,Bangladesh,KZG1
...,...,...,...,...,...,...
1183269,2026-02-08,wepay,bank-transfer,WITHDRAW,Thailand,THKZG2
1183270,2025-12-17,wepay,bank-transfer,WITHDRAW,Thailand,96G1
1183271,2025-12-03,wepay,bank-transfer,WITHDRAW,Thailand,BLG1
1183272,2025-12-29,wepay,bank-transfer,WITHDRAW,Thailand,KZG1


In [97]:
df_withdraw = df_withdraw[df_withdraw["Date"] >= "2025-11-01"]

In [98]:
df_withdraw[["Count", "Total_Net_Amount", "winsorized_total_time_seconds", "Count_03m00s_Below", "Count_03m31s_to_05m00s", "Count_05m00s_to_10m00s", 'Count_10m00s_Above']]

,Count,Total_Net_Amount,winsorized_total_time_seconds,Count_03m00s_Below,Count_03m31s_to_05m00s,Count_05m00s_to_10m00s,Count_10m00s_Above
10,24,55038.0,3299.0,19,0,4,1
13,41,6419.0,7035.0,28,13,0,0
14,53,43522.0,2158.0,53,0,0,0
17,17,3632.0,788.0,17,0,0,0
20,22,24423.0,803.0,22,0,0,0
...,...,...,...,...,...,...,...
1183269,7,9962.0,1868.0,2,2,3,0
1183270,2,350.0,126.0,2,0,0,0
1183271,3,514.0,1627.0,0,1,1,1
1183272,2,4269.0,125.0,2,0,0,0


In [99]:
# 1. Define your columns
group_cols = ["Date", "providerName", "channel_type", "type", "Country", "account_group", "group_re", "Hour"]

sum_cols = [
    "Count", 
    "Total_Net_Amount", 
    "winsorized_total_time_seconds", 
    "Count_03m00s_Below", 
    "Count_03m31s_to_05m00s", 
    "Count_05m00s_to_10m00s", 
    "Count_10m00s_Above"
]

# 2. Perform Group By and Sum
df_grouped = df_withdraw.groupby(group_cols)[sum_cols].sum().reset_index()

# 3. (Optional) Inspect the result
print(df_grouped.head())

        Date providerName channel_type      type Country account_group  \
0 2025-11-30    toppay-mx        clabe  WITHDRAW  Mexico          KZG1   
1 2025-11-30    toppay-mx        clabe  WITHDRAW  Mexico          KZG1   
2 2025-11-30    toppay-mx        clabe  WITHDRAW  Mexico          KZG1   
3 2025-11-30    toppay-mx        clabe  WITHDRAW  Mexico          KZG1   
4 2025-11-30    toppay-mx        clabe  WITHDRAW  Mexico          KZG1   

  group_re           Hour  Count  Total_Net_Amount  \
0      KZO  17:00 - 17:59      1             208.0   
1      KZO  18:00 - 18:59      4             352.0   
2      KZO  19:00 - 19:59      9            4331.0   
3      KZO  20:00 - 20:59      3             200.0   
4      KZO  21:00 - 21:59      5             751.0   

   winsorized_total_time_seconds  Count_03m00s_Below  Count_03m31s_to_05m00s  \
0                          136.0                   1                       0   
1                          630.0                   4                  

In [100]:
import pandas as pd
from datetime import timedelta

# --- 1. PREPARE DATA & INDICES ---
# Assumes df_grouped has [Date, Hour, providerName, etc., Count, NetAmount...]

print("Calculating Week Indices...")
df_grouped['Date'] = pd.to_datetime(df_grouped['Date'])
df_grouped['Week_Index'] = df_grouped['Date'].dt.isocalendar().week.astype(int)

# Helper: Day of Week (0=Mon, 6=Sun)
df_grouped['DayOfWeek'] = df_grouped['Date'].dt.dayofweek

# --- 2. DETERMINE THE CUTOFF (Based on Current Week) ---
print("Identifying Cutoff from Current Week...")

# Identify the Current Week (Highest Index)
current_week_idx = df_grouped['Week_Index'].max()
current_week_data = df_grouped[df_grouped['Week_Index'] == current_week_idx]

if not current_week_data.empty:
    # 1. How far into the week are we? (e.g., Today is Monday = 0)
    cutoff_day_idx = current_week_data['DayOfWeek'].max()
    
    # 2. What is the latest hour on that specific day?
    cutoff_hour = current_week_data.loc[
        current_week_data['DayOfWeek'] == cutoff_day_idx, 'Hour'
    ].max()
    
    print(f" -> Cutoff Point: Day {cutoff_day_idx} (Mon=0) at {cutoff_hour}")
else:
    # Fallback if current week is empty
    cutoff_day_idx = 6 # Sunday
    cutoff_hour = "23:59 - 23:59"
    print(" -> No current data. defaulting to full week.")

# --- 3. CREATE THE 'MARK' COLUMN ---
print("Tagging rows with 'pacing_mark'...")

def get_pacing_mark(row):
    # Rule 1: Always include the current week (it is the reference)
    if row['Week_Index'] == current_week_idx:
        return 'Included'
    
    # Rule 2: For historical weeks, compare Day & Hour
    # A. If the day is EARLIER in the week (e.g. Current is Wed, Row is Mon) -> Keep
    if row['DayOfWeek'] < cutoff_day_idx:
        return 'Included'
    
    # B. If it's the SAME Day, check the Hour
    if row['DayOfWeek'] == cutoff_day_idx:
        if row['Hour'] <= cutoff_hour:
            return 'Included'
        else:
            return 'Excluded' # Same day, but later hour
            
    # C. If the day is LATER in the week (e.g. Current is Mon, Row is Tue) -> Exclude
    if row['DayOfWeek'] > cutoff_day_idx:
        return 'Excluded'

    return 'Excluded'

# Apply the logic
df_grouped['pacing_mark'] = df_grouped.apply(get_pacing_mark, axis=1)

# --- 4. GENERATE LABELS ---
print("Generating Week Labels...")
# Week start (Monday)
df_grouped['Week_Start'] = df_grouped['Date'] - pd.to_timedelta(df_grouped['Date'].dt.dayofweek, unit='D')

# Week end (Sunday)
df_grouped['Week_End'] = df_grouped['Week_Start'] + pd.to_timedelta(6, unit='D')

# Use Week_End for label
df_grouped['Week_Label'] = (
    df_grouped['Week_End'].dt.day.astype(str) + " " +
    df_grouped['Week_End'].dt.strftime('%b %y')
)


# --- 5. AGGREGATE (INCLUDE 'pacing_mark' IN GROUP BY) ---
print("Aggregating...")

final_group_cols = [
    "Week_Label", 
    "Week_Index", 
    "pacing_mark", # <--- CRITICAL: Keep this so you can filter in Looker
    "providerName", "channel_type", "type", "Country", "account_group", "group_re"
]

sum_cols = [
    "Count", "Total_Net_Amount", "winsorized_total_time_seconds", 
    "Count_03m00s_Below", "Count_03m31s_to_05m00s", 
    "Count_05m00s_to_10m00s", "Count_10m00s_Above"
]

df_final = df_grouped.groupby(final_group_cols)[sum_cols].sum().reset_index()

# Sort DESC so Week 49 is top
df_final = df_final.sort_values(by=['Week_Index', 'pacing_mark'], ascending=[False, True])

print(df_final[['Week_Label', 'pacing_mark', 'Count']].head())

# write_df_to_gsheet(df_final)

Calculating Week Indices...
Identifying Cutoff from Current Week...
 -> Cutoff Point: Day 6 (Mon=0) at 23:00 - 23:59
Tagging rows with 'pacing_mark'...
Generating Week Labels...
Aggregating...
    Week_Label pacing_mark  Count
919  28 Dec 25    Included    418
920  28 Dec 25    Included     64
921  28 Dec 25    Included     31
922  28 Dec 25    Included    347
923  28 Dec 25    Included    130


In [101]:
df_final.loc[df_final["Week_Label"].str.contains("26"), "Week_Index"] += 52

In [102]:
df_final.columns = ['Week_Label', 'Week_Index', 'pacing_mark','providerName', 'channel_type', 'type',
       'Country', 'account_group', 'group_re','WD Count', 'Total Amount WD',
       'Avg WD Time', 'Count_03m00s_Below',
       'Count_03m31s_to_05m00s', 'Count_05m00s_to_10m00s',
       'Count_10m00s_Above']
       
# cond1 = df_final["Week_Index"] == (df_final["Week_Index"].max() - 1)
# cond2 = df_final["pacing_mark"] == "Excluded"
# df_final.loc[~(cond1 & cond2), "pacing_mark"] = None

df_final["Week_Index"] = df_final["Week_Index"] - df_final["Week_Index"].max()

# df_final = df_final[df_final["Week_Index"] != 0 ]
# df_final["Week_Index"] = df_final["Week_Index"] + 1
# df_final = df_final[df_final["Week_Index"] >= -5]

In [103]:
df_final

,Week_Label,Week_Index,pacing_mark,providerName,channel_type,type,Country,account_group,group_re,WD Count,Total Amount WD,Avg WD Time,Count_03m00s_Below,Count_03m31s_to_05m00s,Count_05m00s_to_10m00s,Count_10m00s_Above
919,28 Dec 25,-8,Included,aipay,bank-transfer,WITHDRAW,Thailand,96G1,KZP,418,990764.0,68219.0,334,3,47,34
920,28 Dec 25,-8,Included,aipay,bank-transfer,WITHDRAW,Thailand,KZG1,KZP,64,567532.0,35485.0,40,0,5,19
921,28 Dec 25,-8,Included,aipay,bank-transfer,WITHDRAW,Thailand,Others,KZP,31,114278.0,11765.0,18,0,5,8
922,28 Dec 25,-8,Included,aipay,bank-transfer,WITHDRAW,Thailand,THKZG1,KZO,347,2512641.0,165229.0,216,14,32,85
923,28 Dec 25,-8,Included,aipay,bank-transfer,WITHDRAW,Thailand,THKZG2,KZO,130,890719.0,53442.0,72,1,15,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,4 Jan 26,-7,Included,xpay-pk,easypaisa,WITHDRAW,Pakistan,KZG1,KZO,325,1071114.0,22511.0,311,5,5,4
1129,4 Jan 26,-7,Included,xqpay-bd,bkash,WITHDRAW,Bangladesh,KZG1,KZO,1185,1164546.0,70677.0,1166,5,8,6
1130,4 Jan 26,-7,Included,xqpay-bd,nagad,WITHDRAW,Bangladesh,KZG1,KZO,952,864338.0,48122.0,944,3,2,3
1131,4 Jan 26,-7,Included,xqpay-bd,rocket,WITHDRAW,Bangladesh,KZG1,KZO,12,6585.0,2600.0,6,2,3,1


In [104]:
df_total_final = pd.concat([df_final_dep, df_final], axis=0)

In [105]:
df_total_prev = df_total_final[df_total_final["Week_Index"] < 0]
df_total_prev["Week_Index"] = df_total_prev["Week_Index"] + 1

C:\Users\ngoho\AppData\Local\Temp\ipykernel_19620\4263846008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total_prev["Week_Index"] = df_total_prev["Week_Index"] + 1


In [106]:
df_total_prev.columns = ['Week_Label', 'Week_Index', 'pacing_mark', 'providerName',
       'channel_type', 'type', 'Country', 'account_group', 'group_re','DP Count Prev', 'Count Success Prev',
       'Count Error Prev', 'Count Timeout Prev',
       'Total Amount DP Prev', 'Avg DP Time Prev', 'Count_01m30s_Below Prev',
       'Count_01m31s_to_02m00s Prev', 'Count_02m01s_to_03m00s Prev',
       'Count_03m00s_Above Prev', 'WD Count Prev', 'Total Amount WD Prev', 'Avg WD Time Prev',
       'Count_03m00s_Below Prev', 'Count_03m31s_to_05m00s Prev',
       'Count_05m00s_to_10m00s Prev', 'Count_10m00s_Above Prev']

In [107]:
df_total_prev

,Week_Label,Week_Index,pacing_mark,providerName,channel_type,type,Country,account_group,group_re,DP Count Prev,...,Count_01m31s_to_02m00s Prev,Count_02m01s_to_03m00s Prev,Count_03m00s_Above Prev,WD Count Prev,Total Amount WD Prev,Avg WD Time Prev,Count_03m00s_Below Prev,Count_03m31s_to_05m00s Prev,Count_05m00s_to_10m00s Prev,Count_10m00s_Above Prev
1330,28 Dec 25,-7,Included,aipay,qr-code,DEPOSIT,Thailand,96G1,KZP,1306,...,101,59,48,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1331,28 Dec 25,-7,Included,aipay,qr-code,DEPOSIT,Thailand,KZG1,KZP,1222,...,210,168,98,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1332,28 Dec 25,-7,Included,aipay,qr-code,DEPOSIT,Thailand,Others,KZP,1641,...,303,221,76,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1333,28 Dec 25,-7,Included,aipay,qr-code,DEPOSIT,Thailand,THKZG1,KZO,6795,...,1274,1019,410,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1334,28 Dec 25,-7,Included,aipay,qr-code,DEPOSIT,Thailand,THKZG2,KZO,6413,...,1121,853,327,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,4 Jan 26,-6,Included,xpay-pk,easypaisa,WITHDRAW,Pakistan,KZG1,KZO,<NA>,...,<NA>,<NA>,<NA>,325,1071114.0,22511.0,311,5,5,4
1129,4 Jan 26,-6,Included,xqpay-bd,bkash,WITHDRAW,Bangladesh,KZG1,KZO,<NA>,...,<NA>,<NA>,<NA>,1185,1164546.0,70677.0,1166,5,8,6
1130,4 Jan 26,-6,Included,xqpay-bd,nagad,WITHDRAW,Bangladesh,KZG1,KZO,<NA>,...,<NA>,<NA>,<NA>,952,864338.0,48122.0,944,3,2,3
1131,4 Jan 26,-6,Included,xqpay-bd,rocket,WITHDRAW,Bangladesh,KZG1,KZO,<NA>,...,<NA>,<NA>,<NA>,12,6585.0,2600.0,6,2,3,1


In [108]:
index_cols = [
    'Week_Label', 'Week_Index', 'providerName',
    'channel_type', 'type', 'Country', 'account_group', 'group_re'
]

metric_cols = [
    'DP Count',  "Count Success", "Count Error", "Count Timeout", 'Total Amount DP', 'Avg DP Time',
    'Count_01m30s_Below', 'Count_01m31s_to_02m00s',
    'Count_02m01s_to_03m00s', 'Count_03m00s_Above',
    'WD Count', 'Total Amount WD', 'Avg WD Time',
    'Count_03m00s_Below', 'Count_03m31s_to_05m00s',
    'Count_05m00s_to_10m00s', 'Count_10m00s_Above'
]

df_grouped_final = (
    df_total_final
    .groupby(index_cols)[metric_cols]
    .sum()
    .reset_index()
)

In [109]:
df_grouped_final = df_grouped_final.sort_values(by=['Week_Index', 'type'], ascending=[False, True])

In [110]:
index_cols = [
    'Week_Label', 'Week_Index', 'providerName',
    'channel_type', 'type', 'Country', 'account_group', "group_re"
]

metric_cols_prev = [
    'DP Count Prev',  "Count Success Prev", "Count Error Prev", "Count Timeout Prev", 'Total Amount DP Prev', 'Avg DP Time Prev',
    'Count_01m30s_Below Prev', 'Count_01m31s_to_02m00s Prev',
    'Count_02m01s_to_03m00s Prev', 'Count_03m00s_Above Prev',
    'WD Count Prev', 'Total Amount WD Prev', 'Avg WD Time Prev',
    'Count_03m00s_Below Prev', 'Count_03m31s_to_05m00s Prev',
    'Count_05m00s_to_10m00s Prev', 'Count_10m00s_Above Prev'
]

df_grouped_prev = (
    df_total_prev
    .groupby(index_cols)[metric_cols_prev]
    .sum()
    .reset_index()
)

In [111]:
df_grouped_final

,Week_Label,Week_Index,providerName,channel_type,type,Country,account_group,group_re,DP Count,Count Success,...,Count_01m31s_to_02m00s,Count_02m01s_to_03m00s,Count_03m00s_Above,WD Count,Total Amount WD,Avg WD Time,Count_03m00s_Below,Count_03m31s_to_05m00s,Count_05m00s_to_10m00s,Count_10m00s_Above
1748,22 Feb 26,0,aipay,qr-code,DEPOSIT,Thailand,KZG1,KZP,438,346,...,103,67,34,0,0.0,0.0,0,0,0,0
1749,22 Feb 26,0,aipay,qr-code,DEPOSIT,Thailand,Others,KZP,656,494,...,143,86,52,0,0.0,0.0,0,0,0,0
1750,22 Feb 26,0,aipay,qr-code,DEPOSIT,Thailand,THKZG1,KZO,2403,2052,...,565,352,173,0,0.0,0.0,0,0,0,0
1751,22 Feb 26,0,aipay,qr-code,DEPOSIT,Thailand,THKZG2,KZO,2278,1852,...,477,309,207,0,0.0,0.0,0,0,0,0
1752,22 Feb 26,0,akepay-mx,clabe,DEPOSIT,Mexico,KZG1,KZO,14,13,...,0,3,1,0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2551,30 Nov 25,-12,toppay-mx,clabe,WITHDRAW,Mexico,KZG1,KZO,0,0,...,0,0,0,35,9415.0,5684.0,29,6,0,0
2554,30 Nov 25,-12,zenpay,pix,WITHDRAW,Brazil,KZG1,KZO,0,0,...,0,0,0,7,307.0,448.0,7,0,0,0
1942,23 Nov 25,-13,hspay,bank-transfer-native,DEPOSIT,Thailand,THKZG2,KZO,1,1,...,0,0,1,0,0.0,0.0,0,0,0,0
1192,16 Nov 25,-14,hspay,bank-transfer-native,DEPOSIT,Thailand,WDB1,KZP,1,1,...,0,0,1,0,0.0,0.0,0,0,0,0


In [112]:
df_grouped_prev

,Week_Label,Week_Index,providerName,channel_type,type,Country,account_group,group_re,DP Count Prev,Count Success Prev,...,Count_01m31s_to_02m00s Prev,Count_02m01s_to_03m00s Prev,Count_03m00s_Above Prev,WD Count Prev,Total Amount WD Prev,Avg WD Time Prev,Count_03m00s_Below Prev,Count_03m31s_to_05m00s Prev,Count_05m00s_to_10m00s Prev,Count_10m00s_Above Prev
0,1 Feb 26,-2,aipay,bank-transfer,WITHDRAW,Thailand,96G1,KZP,0,0,...,0,0,0,34,544192.0,3896.0,31,1,1,1
1,1 Feb 26,-2,aipay,bank-transfer,WITHDRAW,Thailand,KZG1,KZP,0,0,...,0,0,0,121,570803.0,29743.0,96,12,4,9
2,1 Feb 26,-2,aipay,bank-transfer,WITHDRAW,Thailand,Others,KZP,0,0,...,0,0,0,118,667336.0,53750.0,94,9,4,11
3,1 Feb 26,-2,aipay,bank-transfer,WITHDRAW,Thailand,THKZG1,KZO,0,0,...,0,0,0,840,4103224.0,212921.0,665,71,45,59
4,1 Feb 26,-2,aipay,bank-transfer,WITHDRAW,Thailand,THKZG2,KZO,0,0,...,0,0,0,481,2572528.0,179159.0,371,42,27,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,8 Feb 26,-1,ydpay-mx,clabe,DEPOSIT,Mexico,MXNKZG1,KZO,812,635,...,118,107,91,0,0.0,0.0,0,0,0,0
3229,8 Feb 26,-1,ydpay-mx,clabe,WITHDRAW,Mexico,KZG1,KZO,0,0,...,0,0,0,2010,273935.0,311712.0,1859,133,3,15
3230,8 Feb 26,-1,ydpay-mx,clabe,WITHDRAW,Mexico,MXNKZG1,KZO,0,0,...,0,0,0,334,25175.0,50461.0,312,18,2,2
3231,8 Feb 26,-1,zenpay,pix,DEPOSIT,Brazil,KZG1,KZO,1844,1490,...,72,54,27,0,0.0,0.0,0,0,0,0


In [113]:
df_grouped_concat = df_grouped_final.merge(df_grouped_prev[['Week_Index', 'providerName',
    'channel_type', 'type', 'Country', 'account_group', 'group_re','DP Count Prev',  "Count Success Prev", "Count Error Prev", "Count Timeout Prev",'Total Amount DP Prev', 'Avg DP Time Prev',
    'Count_01m30s_Below Prev', 'Count_01m31s_to_02m00s Prev',
    'Count_02m01s_to_03m00s Prev', 'Count_03m00s_Above Prev',
    'WD Count Prev', 'Total Amount WD Prev', 'Avg WD Time Prev',
    'Count_03m00s_Below Prev', 'Count_03m31s_to_05m00s Prev',
    'Count_05m00s_to_10m00s Prev', 'Count_10m00s_Above Prev']], how = 'outer', on = ['Week_Index', 'providerName',
       'channel_type', 'type', 'Country', 'account_group', 'group_re'])

In [114]:
list_data = df_grouped_concat[df_grouped_concat["Week_Label"].notna()][["Week_Label", "Week_Index"]].drop_duplicates()

In [115]:
df_grouped_concat = df_grouped_concat[[i for i in df_grouped_concat.columns if "Week_Label" not in i]].merge(list_data,  how = "left")

In [116]:
df_grouped_concat["Week_Label"] = df_grouped_concat["Week_Label"].astype(str)

In [117]:
df_grouped_concat.groupby(['Week_Label', 'Week_Index']).agg({"WD Count" : "sum"
                                                             , "WD Count Prev": "sum"}).reset_index().sort_values(by=['Week_Index'], ascending=[False])

,Week_Label,Week_Index,WD Count,WD Count Prev
7,22 Feb 26,0,47663,634008
3,15 Feb 26,-1,634008,615009
14,8 Feb 26,-2,615009,593086
0,1 Feb 26,-3,593086,528680
9,25 Jan 26,-4,528680,525326
5,18 Jan 26,-5,525326,485825
1,11 Jan 26,-6,485825,504189
12,4 Jan 26,-7,504189,495821
10,28 Dec 25,-8,495821,432244
6,21 Dec 25,-9,432244,400276


In [118]:
df_grouped_concat = df_grouped_concat[["Week_Label"] + [i for i in df_grouped_concat.columns if i != "Week_Label"]]

In [119]:
df_grouped_concat = df_grouped_concat[df_grouped_concat["Week_Index"] >= -6]

In [120]:
df_grouped_concat["group_re"] = df_grouped_concat["group_re"].str.replace("KZG", "KZO")

In [121]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

# --- 1. CONFIGURATION ---
CREDENTIALS_FILE = 'plexiform-armor-481509-h4-99dffa062794.json'
SHEET_ID = '1Nq9u4bg0tvLnUutVh2TcxXOxe-G2E65kxy_pbJ8pce4'
WORKSHEET_NAME = 'WoW Data'

def write_df_to_gsheet(dataframe):
    # --- 2. AUTHENTICATION ---
    # Define the scope (permissions) required
    scopes = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]
    
    # Authenticate using the JSON key file
    credentials = Credentials.from_service_account_file(
        CREDENTIALS_FILE, 
        scopes=scopes
    )
    gc = gspread.authorize(credentials)

    # --- 3. CONNECT TO SHEET ---
    print(f"Opening sheet ID: {SHEET_ID}...")
    sh = gc.open_by_key(SHEET_ID)
    
    try:
        worksheet = sh.worksheet(WORKSHEET_NAME)
    except gspread.WorksheetNotFound:
        print(f"Worksheet '{WORKSHEET_NAME}' not found. Creating it...")
        worksheet = sh.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

    # --- 4. WRITE DATA ---
    print("Clearing old data...")
    worksheet.clear()  # Removes old data so rows don't overlap
    
    print(f"Writing {len(dataframe)} rows to Google Sheets...")
    # 'set_with_dataframe' handles headers and NaN values automatically
    set_with_dataframe(worksheet, dataframe)
    
    print("Done!")

# --- Usage ---
if __name__ == "__main__":
    # Assuming 'df' exists from your previous code
    # If not, uncomment the line below to test with dummy data:
    # df = pd.DataFrame({'Date': ['2023-01-01'], 'Amount': [100]})
    
    write_df_to_gsheet(df_grouped_concat.sort_values(by=['Week_Index'], ascending=[False]))

Opening sheet ID: 1Nq9u4bg0tvLnUutVh2TcxXOxe-G2E65kxy_pbJ8pce4...
Clearing old data...
Writing 2390 rows to Google Sheets...
Done!


In [122]:
df_deposit[df_deposit["Country"] == "Bangladesh"]

,Date,providerKey,method,channel_type,type,reqCurrency,Country,status,Hour,Count,...,Count_05m00s_to_10m00s,Count_10m00s_Above,providerName,channel_main,brand,account_group,group_re,Count Success,Count Error,Count Timeout
5,2026-01-24,wingpay-bd,wingpay-bd/bkash-qr,bkash-qr,DEPOSIT,BDT,Bangladesh,timeout,21:00 - 21:59,98,...,0,0,wingpay-bd,bkash,ADDA7,KZG1,KZO,<NA>,<NA>,98
6,2025-12-31,xqpay-bd,xqpay-bd/bkash,bkash,DEPOSIT,BDT,Bangladesh,timeout,01:00 - 01:59,21,...,0,0,xqpay-bd,bkash,ADDA7,KZG1,KZO,<NA>,<NA>,21
7,2026-02-01,tgpay-bd,tgpay-bd/bkash,bkash,DEPOSIT,BDT,Bangladesh,timeout,14:00 - 14:59,61,...,0,0,tgpay-bd,bkash,ADDA7,KZG1,KZO,<NA>,<NA>,61
9,2026-01-26,tgpay-bd,tgpay-bd/nagad,nagad,DEPOSIT,BDT,Bangladesh,completed,06:00 - 06:59,7,...,0,0,tgpay-bd,nagad,BDPOP,KZG1,KZO,7,<NA>,<NA>
11,2026-01-16,xpay-bd,xpay-bd/bkash,bkash,DEPOSIT,BDT,Bangladesh,timeout,19:00 - 19:59,32,...,0,0,xpay-bd,bkash,ADDA7,KZG1,KZO,<NA>,<NA>,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980990,2026-01-25,wingpay-bd,wingpay-bd/bkash-qr,bkash-qr,DEPOSIT,BDT,Bangladesh,completed,22:00 - 22:59,1,...,0,0,wingpay-bd,bkash,BDPOP,KZG1,KZO,1,<NA>,<NA>
980993,2026-01-26,dpp-bd,dpp-bd/rocket,rocket,DEPOSIT,BDT,Bangladesh,timeout,12:00 - 12:59,1,...,0,0,dpp-bd,rocket,ADDA7,KZG1,KZO,<NA>,<NA>,1
981000,2025-12-20,xqpay-bd,xqpay-bd/nagad,nagad,DEPOSIT,BDT,Bangladesh,completed,12:00 - 12:59,2,...,0,0,xqpay-bd,nagad,ADDA7,KZG1,KZO,2,<NA>,<NA>
981007,2026-01-19,None,None,None,DEPOSIT,BDT,Bangladesh,completed,14:00 - 14:59,1,...,0,0,None,None,ADDA7,KZG1,KZO,1,<NA>,<NA>


In [123]:
df_grouped_concat[(df_grouped_concat["type"] == "DEPOSIT") & 
                  (df_grouped_concat["Country"] == "Bangladesh") &
                  (df_grouped_concat["Week_Label"] == "4 Jan 26")]

,Week_Label,Week_Index,providerName,channel_type,type,Country,account_group,group_re,DP Count,Count Success,...,Count_01m31s_to_02m00s Prev,Count_02m01s_to_03m00s Prev,Count_03m00s_Above Prev,WD Count Prev,Total Amount WD Prev,Avg WD Time Prev,Count_03m00s_Below Prev,Count_03m31s_to_05m00s Prev,Count_05m00s_to_10m00s Prev,Count_10m00s_Above Prev


In [124]:
df_grouped_concat

,Week_Label,Week_Index,providerName,channel_type,type,Country,account_group,group_re,DP Count,Count Success,...,Count_01m31s_to_02m00s Prev,Count_02m01s_to_03m00s Prev,Count_03m00s_Above Prev,WD Count Prev,Total Amount WD Prev,Avg WD Time Prev,Count_03m00s_Below Prev,Count_03m31s_to_05m00s Prev,Count_05m00s_to_10m00s Prev,Count_10m00s_Above Prev
1513,11 Jan 26,-6,aipay,bank-transfer,WITHDRAW,Thailand,96G1,KZP,0,0,...,0,0,0,171,1532048.0,35661.0,150,1,5,15
1514,11 Jan 26,-6,aipay,bank-transfer,WITHDRAW,Thailand,KZG1,KZP,0,0,...,0,0,0,129,768158.0,48198.0,86,7,6,30
1515,11 Jan 26,-6,aipay,bank-transfer,WITHDRAW,Thailand,Others,KZP,0,0,...,0,0,0,362,2388992.0,106297.0,298,11,14,39
1516,11 Jan 26,-6,aipay,bank-transfer,WITHDRAW,Thailand,THKZG1,KZO,0,0,...,0,0,0,870,6742950.0,318001.0,577,57,56,180
1517,11 Jan 26,-6,aipay,bank-transfer,WITHDRAW,Thailand,THKZG2,KZO,0,0,...,0,0,0,1318,8486674.0,453562.0,1059,45,55,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3898,22 Feb 26,0,ydpay-mx,clabe,DEPOSIT,Mexico,MXNKZG1,KZO,<NA>,<NA>,...,3,1,2,0,0.0,0.0,0,0,0,0
3899,22 Feb 26,0,ydpay-mx,clabe,WITHDRAW,Mexico,KZG1,KZO,<NA>,<NA>,...,0,0,0,76,19936.0,8906.0,75,0,1,0
3900,22 Feb 26,0,ydpay-mx,clabe,WITHDRAW,Mexico,MXNKZG1,KZO,<NA>,<NA>,...,0,0,0,23,4530.0,2784.0,23,0,0,0
3901,22 Feb 26,0,zenpay,pix,DEPOSIT,Brazil,KZG1,KZO,<NA>,<NA>,...,64,33,29,0,0.0,0.0,0,0,0,0
